In [ ]:
! pip install RDKit openmm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 85.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for openmm: filename=openmm-0.2-py3-none-any.whl size=61164 sha256=4f0747584d27b00c02a961fe95a525175a4716995a1410cd2ac672373725e817
  Stored in directory: /root/.cache/pip/wheels/7b/38/69/107dcff07bcbf69542be3a2df57e62966655d78352c3d05757
Successfully built openmm


In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem

# Load the SMILES strings from the input.csv file
df = pd.read_csv('input.csv')
#df = df.drop([10, 13, 28]) # Drops the first three rows
smiles_list = df['Smiles'].tolist()

df


,Smiles
0,OCC1OC(OC2CC3=C(OC2C2=CC(O)=C(O)C=C2)C=C(O)C=C...
1,O[C@H]1CC2=C(O)C=C(O)C=C2O[C@@H]1C1=CC(O)=C(O)...
2,OC1=CC(O)=C2C[C@H](OC(=O)C3=CC(O)=C(O)C(O)=C3)...
3,O[C@H]1CC2=C(O)C=C(O)C=C2O[C@H]1C1=CC=C(O)C(O)=C1
4,O[C@H]1CC2=C(O)C=C(O)C=C2O[C@H]1C1=CC(OS(O)(=O...
...,...
573,COC1=CC(=CC(OC)=C1O)C1=[O+]C2=C3C(OC(=CC3=C1O[...
574,O[C@H]1O[C@@H](OC2=CC(O)=C(C(=O)CCC3=CC=C(O)C=...
575,[H][C@]1(O[C@H](CO)[C@@H](O)[C@H](O)[C@H]1O)C1...
576,OC[C@H]1O[C@H]([C@H](O)[C@@H](O)[C@@H]1O)C1=C2...


In [ ]:
mols = []
for smiles in smiles_list:
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        mols.append(mol)
    else:
        print(f'Could not create molecule from SMILES string: {smiles}')

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem

# Initialize a list to store the indices of failed conversions
failed_indices = []

# Convert the SMILES strings to RDKit molecules
mols = [Chem.MolFromSmiles(smiles) for smiles in smiles_list]

# Iterate through the molecules
for i, mol in enumerate(mols):
    try:
        # Add explicit hydrogen atoms to the molecule
        mol = Chem.AddHs(mol)

        # Minimize the structure of the molecule with hydrogen atoms
        AllChem.EmbedMolecule(mol)
        AllChem.UFFOptimizeMolecule(mol)

        # Save the minimized structure as a PDB file
        Chem.MolToPDBFile(mol, f'input_{i}.pdb')
    except Exception as e:
        # Handle the exception by reporting the index and moving to the next conversion
        print(f"Failed to convert structure at index {i}: {str(e)}")
        failed_indices.append(i)

# At this point, the "failed_indices" list contains the indices of failed conversions
# You can use this list for further analysis or to identify problematic SMILES strings.


Failed to convert structure at index 192: Bad Conformer Id
Failed to convert structure at index 195: Bad Conformer Id
Failed to convert structure at index 196: Bad Conformer Id
Failed to convert structure at index 197: Bad Conformer Id
Failed to convert structure at index 198: Bad Conformer Id
Failed to convert structure at index 201: Bad Conformer Id
Failed to convert structure at index 203: Bad Conformer Id
Failed to convert structure at index 463: Bad Conformer Id


In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem

# Initialize a list to store the indices of failed conversions
failed_indices = []

# Convert the SMILES strings to RDKit molecules
mols = [Chem.MolFromSmiles(smiles) for smiles in smiles_list]

# Iterate through the molecules
for i, mol in enumerate(mols):
    try:
        # Add explicit hydrogen atoms to the molecule
        mol = Chem.AddHs(mol)

        # Minimize the structure of the molecule with hydrogen atoms
        AllChem.EmbedMolecule(mol)
        AllChem.UFFOptimizeMolecule(mol)

        # Save the minimized structure as a PDB file
        output_filename = f'input_{i}.pdb'
        Chem.MolToPDBFile(mol, output_filename)
    except Exception as e:
        # Handle the exception by reporting the index
        print(f"Failed to convert structure at index {i}: {str(e)}")
        failed_indices.append(i)

# At this point, the "failed_indices" list contains the indices of failed conversions
# You can use this list for further analysis or to identify problematic SMILES strings.


Failed to convert structure at index 192: Bad Conformer Id
Failed to convert structure at index 195: Bad Conformer Id
Failed to convert structure at index 196: Bad Conformer Id
Failed to convert structure at index 197: Bad Conformer Id
Failed to convert structure at index 198: Bad Conformer Id
Failed to convert structure at index 201: Bad Conformer Id
Failed to convert structure at index 203: Bad Conformer Id


In [ ]:
import os
import zipfile

# Zip all the existing minimized PDB files
with zipfile.ZipFile('minimized_structures.zip', 'w') as myzip:
    for i in range(len(mols)):
        pdb_file = f'input_{i}.pdb'
        if os.path.exists(pdb_file):
            myzip.write(pdb_file)
        else:
            print(f"Warning: File {pdb_file} not found, skipping.")
